In [6]:
from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource, PluginSetting
sales = O9DataLake.register("SalesActual" ,DataSource.LIVEFRAME,ResourceType.LIVEFRAME, plugin_setting = PluginSetting.Inputs)
output = O9DataLake.register("output",data_source = DataSource.LS,entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.Outputs)

In [7]:
O9DataLake.inputs

{'SalesActual': {'name': 'SalesActual',
  'resource_type': <ResourceType.LIVEFRAME: 'liveframe'>,
  'data_source': <DataSource.LIVEFRAME: 'liveframe'>,
  'query': 'SELECT ([Time].[Week].filter(#.[Week$InActive] in { false }) * {Measure.[AIPActual]} * [Item].[Planning Item].filter(#.[Planning Item$InActive] in { false }) * [Location].[Location].filter(#.Name in { "3", "4", "5", "6", "8" }).relatedmembers([Location]).filter(#.[Location$InActive] in { false })) on row, () on column include lcid 1033, cellproperties {LOCK, BG_COLOR, FG_COLOR} where {[Item].[Item].filter(#.Name in { "273940", "547370", "696820", "759830", "838179" }).filter(#.[Item$InActive] in { false }) , [Time].[Year].filter(#.Name in { "2021", "2022", "2023" }).filter(#.[Year$InActive] in { false }) , [Version].[Version Name].filter(#.Name in { "CurrentWorkingView" }).filter(#.[Version Name$InActive] in { false })} ORDERBY [Time].[Week].[Key] ASC ;',
  'LiveFrameReportId': '8ef7dcfe-8a32-472b-83d3-9129d904b332',
  'std_

In [8]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import logging

from o9_flow.O9Flow import O9Flow
o9flow = O9Flow()
logger = logging.getLogger('o9_logger')

logger.info("sample plugin template")
logger.info('o9flow initialising')

df = sales

data = df.rename(columns={'Time.[Week]': 'Week', 'Item.[Planning Item]': 'Item', 'Location.[Location]':'Location'})

data['Week'] = pd.to_datetime(data['Week'])
data.set_index('Week', inplace=True)

items = data['Item'].unique()
locations = data['Location'].unique()

predictions_df = pd.DataFrame(columns=['Item', 'Location', 'Week', 'Predicted Sales'])



logger.info(data.head())

for item in items:
    for location in locations:
        # Filter data for the specific item and location
        subset_data = data[(data['Item'] == item) & (data['Location'] == location)]
        if subset_data.empty != True:
            item_location_sales = subset_data['AIPActual']

        # Calculate Simple Moving Average
            window_size = 3  # Adjust the window size as needed
            sma = item_location_sales.rolling(window=window_size).mean()

            # Make predictions for the next 10 weeks (assuming a constant average)
            average_prediction = sma.iloc[-1]   # Use the last calculated average

            # Create a DataFrame with the predictions
            predictions_subset = pd.DataFrame({
                'Item': [item] * 10,
                'Location': [location] * 10,
                'Week': pd.date_range(start=item_location_sales.index[-1] + pd.DateOffset(1), periods=10, freq='W'),
                'Predicted Sales': [average_prediction] * 10
            })

            # Append the predictions to the overall DataFrame
            predictions_df = predictions_df.append(predictions_subset, ignore_index=True)

            
            predictions_df['Week'] = pd.to_datetime(predictions_df['Week']).dt.strftime('%d-%b-%y')

o9flow.log_params(params= {'param_A':8, 'param_B':1.5, 'param_C':'Testing'})
o9flow.log_metrics(metrics={'plugin_feature_A': 4.99, 'plugin_feature_B': 3.00}, step=11)
            
output = predictions_df.rename(columns={'Week':'Time.[Week]', 'Item':'Item.[Planning Item]', 'Location':'Location.[Location]'})
output['Version.[Version Name]'] = 'CurrentWorkingView'
List = ['Version.[Version Name]','Item.[Planning Item]','Location.[Location]','Time.[Week]','Predicted Sales']
output = output[List]

logger.info(output.head())


find: ‘/usr/hdp’: No such file or directory


O9Flow Service is running now...
2024-01-22 13:54:50,058 - o9_logger - INFO  - sample plugin template
2024-01-22 13:54:50,059 - o9_logger - INFO  - o9flow initialising
2024-01-22 13:54:50,072 - o9_logger - INFO  -               Item  Location  AIPActual
Week                                   
2021-01-03  273940         4     2293.0
2021-01-03  273940         6     2936.0
2021-01-03  273940         8     7402.0
2021-01-03  547370         4     4001.0
2021-01-03  547370         6     4907.0
2024-01-22 13:54:50,436 - o9_logger - INFO  -   Version.[Version Name] Item.[Planning Item] Location.[Location] Time.[Week]  \
0     CurrentWorkingView               273940                   4   05-Feb-23   
1     CurrentWorkingView               273940                   4   12-Feb-23   
2     CurrentWorkingView               273940                   4   19-Feb-23   
3     CurrentWorkingView               273940                   4   26-Feb-23   
4     CurrentWorkingView               273940          

In [9]:
O9DataLake.put('output', output)